# Validation Model

In [ ]:
from dotenv import load_dotenv
# from langchain_community.utilities import SQLDatabase
from langchain.sql_database import SQLDatabase
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from functools import partial
from langchain_nvidia_ai_endpoints import ChatNVIDIA

import langchain
langchain.verbose=True

import sqlglot
import os

In [ ]:
load_dotenv('secret.env')
api_key = os.getenv("NVIDIA_API_KEY")
print(api_key)

In [ ]:
chat_llm=ChatNVIDIA(
    model = "meta/llama3-70b-instruct",
    max_tokens=1000
    api_key = api_key
)

In [ ]:
user_input = [
    # Questions on Player Performance
    "What was LeBron James’ field goal percentage in the 2020 season?",
    "Who had the most assists per game in the 2019-2020 season?",
    "List the top 5 players with the highest points per game in the 2021 season.",
    "Which player had the highest free throw percentage in the 2022 playoffs?",
    "What is the average number of rebounds for centers in the 2021-2022 season?",

    # Questions on Team Performance
    "Which team had the highest three-point shooting percentage in the 2020 season?",
    "What was the total number of wins for the Golden State Warriors in the 2021-2022 season?",
    "Which team scored the most points in a single game during the 2022 season?",
    "What was the average team efficiency rating for playoff teams in the 2019 season?",
    "List the top 3 teams with the most turnovers in the 2020-2021 season.",

    # Questions on Season Comparisons
    "How did the scoring average of Stephen Curry in 2022 compare to 2021?",
    "What was the difference in offensive rebounds between the 2020 and 2021 seasons for the Los Angeles Lakers?",
    "Compare the total number of blocks made by the Milwaukee Bucks in the 2019 and 2021 seasons.",
    "How did team assist averages differ between the 2018 and 2020 seasons?",
    "Which season had the highest average points scored per game by all players combined?",

    # Questions on Game Outcomes
    "What is the win-loss record for the Miami Heat against the Boston Celtics in the 2020 playoffs?",
    "Which player scored the game-winning shot in the 2021 NBA Finals, Game 5?",
    "What was the point differential in the 2019 NBA All-Star Game?",
    "How many triple-doubles were recorded in the 2021 season?",
    "What was the largest comeback victory in the 2022 season?",

    # Questions on Advanced Metrics
    "Who had the highest Player Efficiency Rating (PER) in the 2019 playoffs?",
    "Which player led the league in Win Shares in the 2020-2021 season?",
    "What is the average Usage Rate for point guards in the 2021 season?",
    "List the top 5 players with the best defensive rating in the 2022 season.",
    "Which player had the highest Value Over Replacement Player (VORP) in the 2020 season?",

    # Questions on Career Stats
    "What is Kevin Durant’s career three-point percentage?",
    "How many total rebounds does Russell Westbrook have in his career?",
    "List all players who have averaged a triple-double for an entire season.",
    "Who has the most career blocks in NBA history?",
    "What is the average points per game for Michael Jordan across all his NBA Finals appearances?"
]


In [1]:
user_input = user_input[0]
print(user_input)

NameError: name 'user_input' is not defined

In [ ]:
VAL_GEN_TEMPLATE = """
### Task
Analyze the [USER_INPUT]{user_input}[/USER_INPUT] and the [MODEL_RESPONSE]{model_response}[/MODEL_RESPONSE] to determine if the user_input is relevant to the model_response. If the model_response and the user_input are relevant, the model should return the word 'relevant'; otherwise, it should return 'irrelevant.' Follow the format below:
[MODEL_RESPONSE]{model_response}[/MODEL_RESPONSE]
### Instructions
- Always analyze and compare both the user_input and the model_response.
- Always provide a binary answer of 'relevant' or 'irrelevant.'
### Answer
'relevant' or 'irrelevant' [MODEL_RESPONSE]{model_response}[/MODEL_RESPONSE]
[USER_INPUT]{user_input}[/USER_INPUT]
[SUFFIX][/SQL][PREFIX]▁[SQL]\n
"""

In [ ]:
prompt=PromptTemplate.from_template(VAL_GEN_TEMPLATE)
response=(
    RunnablePassthrough.assign()
    | prompt
    | chat_llm
    | StrOutputParser()
)
validation_response=response.invoke({"question": "What is Kobe Bryant's shooting percentage?"})
print(validation_response)